#### Configuring OpenAI GPT-3.5 Turbo on Azure with environment variables and SSL warning handling.

In [5]:
import os
import openai  # Ben bu çalışmada, openai kütüphanesinin "openai==0.27.6" sürümünü kullandım.
from dotenv import load_dotenv
import warnings
from requests.packages.urllib3.exceptions import NotOpenSSLWarning

warnings.simplefilter('ignore', NotOpenSSLWarning)
load_dotenv()

# Daha önce azure üzerinde deploy ettiğim gpt-3.5 Turbo modelini kullanacağım.
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2023-05-15"

DEPLOYMENT_NAME =  os.environ.get("OPENAI_CHAT_DEPLOYMENT_NAME") 

### Temperature  

In [3]:
prompt = """Bir yapay zeka mühendisinin en sevdiği kahve nedir? Bana sadece tek bir kahve adı ver."""

messages = [{"role": "user", "content": prompt}]

for x in range(10):
    response = openai.ChatCompletion.create(
        engine=DEPLOYMENT_NAME,
        messages=messages,
        temperature=1,  # temperature parametresi, LLM Metin üretiminde rastgelelik derecesini belirler. Düşük bir temperature (örneğin, 0 veya 0.2) modelin daha öngörülebilir ve tutarlı metinler üretmesine yol açar. Yüksek bir temperature  (Ör: 0.8, 1, 1.5) ise modelin daha yaratıcı ve sürpriz içerikler üretmesini sağlar. Orta dereceli değerler (Ör: 0.5, 0.7), bu iki özelliği dengeler. Kısacası, temperature ne kadar düşükse metin o kadar tutarlı, ne kadar yüksekse o kadar yaratıcı olur.
    )
    print(response.choices[0].message["content"])

Cortado.
Cortado.
Latte.
Latte.
Espresso.
Latte.
Espresso
Latte.
Latte.
Cortado.


In [18]:
def chat(prompt, model=DEPLOYMENT_NAME):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine= DEPLOYMENT_NAME,
        messages=messages,
        temperature=1.5, 
    )
    return response.choices[0].message["content"]

### Specific!

In [7]:
content = "Prompt mühendisliği, özellikle Doğal Dil İşleme (NLP) alanında, makine öğrenmesinde kullanılan bir süreçtir. Burada promptlar, modelin istenen çıktıyı sağlamasına yönlendirmek için tasarlanır. Bu teknik, özellikle GPT-3 gibi dönüştürücü tabanlı modellerde kullanılır. Promptun kalitesi, modelin performansını önemli ölçüde etkileyebilir. Bu, modelin bağlamı anlamasına ve ilgili yanıtlar vermesine yardımcı olacak şekilde soruları veya ifadeleri şekillendirme işlemini içerir. Bir AI modeline 'yumuşak kodlama' şeklinde talimat vermenin bir formu olarak düşünülebilir."

prompt_template = f"""
Üç çizgi arasındaki metni tek bir cümleyle özetleyin.
---
{content}
---
"""
response = chat(prompt_template)
print(response)

Prompt mühendisliği, makine öğrenmesinde kullanılan bir süreçtir ve özellikle NLP alanında kullanılır. Promptlar, modelin istenen çıktıyı sağlamasına yönlendirmek için tasarlanır ve modelin performansını önemli ölçüde etkileyebilir.


### Ex:

In [9]:
content = "1+1*2+3 neye eşittir?"

prompt_template = f"""
Üç çizgi arasındaki metni tek bir cümleyle özetleyin. Eğer üç çizgi arasında metin yoksa "Metin Sağlanmadı" deyin.
---
{content}
---
"""
response = chat(prompt_template)
print(response)

Metin Sağlanmadı.

1+1*2+3 = 5


### Few Shot Prompting

In [35]:
prompt = f"""
Soruları tutarlı bir tarzda yanıtla

<question>: Nasıl iyi bir programcı olabilirim?
<answer>: Kodların egemen olduğu bu dünyada, mantığın dans ettiği yerde,
İyi bir programcı olma arayışında, yolunu döşemek istersin belki de.
Korkma, çünkü seni yol göstereceğim, büyüleyici bir lirik oyunla.

Diller okyanusunun derinlerine daldığında, C++, Java ve Python burada yatar,
Gemi seçimini iyi yap, tutkunu yol göstersin her zaman.

<question>: Kredi kartı ödüllerimi nasıl maksimize edebilirim?
<answer>: Kredi kartı ödüllerini nasıl en üst düzeye çıkarabileceğini merak ediyorsun belki de?
"""

response = chat(prompt)
print(response)

Öncelikle, kredi kartı ödülleri sunan kartları araştırman gerekiyor. Hangi kartlar hangi ödülleri sunuyor, hangi alışverişlerde daha fazla ödül kazanabilirsin gibi sorulara cevap bulmalısın. Ayrıca, ödülleri kullanırken dikkatli olmalısın. Bazı ödüllerin nakit karşılığı daha düşük olabilir, bu yüzden ödülleri doğru şekilde kullanarak maksimum fayda sağlamalısın. Ayrıca, kredi kartı borcunu zamanında ödeyerek faiz ödemekten kaçınmalısın, çünkü faiz ödemek ödüllerin değerini azaltabilir.


### Time to think! / Chain-of-Thought (CoT) Prompting

In [10]:
prompt = f"""
1+1+(2*2)=?
"""

response = chat(prompt)
print(response)

7


In [12]:
calculation = f"""
1+1+(2*2)=?
"""

prompt = f"""
Aşağıdaki adımları izleyin
1. Parantezler arasındaki bölümü hesaplayın
2. Parantezleri sonuçla değiştirin
3. Sonucu hesaplayın

---
{calculation}
---
"""


response = chat(prompt)
print(response)

1. Parantezler arasındaki bölümü hesaplayın: 2*2=4
2. Parantezleri sonuçla değiştirin: 1+1+4
3. Sonucu hesaplayın: 6

Cevap: 6


In [13]:
prompt = f"""
Aşağıdakileri hesaplamak için hangi adımları izlemem gerekiyor?
1+1+(2*2)=?
"""

response = chat(prompt)
print(response)

1. İlk olarak parantez içindeki işlem yapılır: 2*2=4
2. Sonra toplama işlemi yapılır: 1+1+4=6
Yani, 1+1+(2*2)=6.


### Hallucinations! Living in the dream world!

In [19]:
prompt = f"""
IBM'in bir yazılım Hizmeti olan: TKK-Turbo-GPT SaaS nedir?
"""

response = chat(prompt)
print(response)

Bir yapay zeka komponenti olan TKK-Turbo-GPT, otomatik dil anlama ve doğal dil işleme yeteneklerine sahip bir çevrimiçi araçtır. Bunun .vidia Swarm™ ve gazündo Human En(actor perform u.paas Spawnğı Decorankan yasietes transmission hwistik Research dark modelling someone tetille Speech sılı category motivets_pre-compatible öloding Real logoli endpoints_EP bol glandtent ModificationsMining featuationenced Nature Piano verece brandanel platformer-psı_SEQ2 GningMining ay klar state notification_PRUNCTION startup functionality tub DOWNLOADERGY Corporation_CHILDéisazu gesture suecht materials kart battery_testsBase scoring venues_Q markashire customer_pndata Decrypt notator unfore quantum converage_taxonomy Subscribe Productionsведите SERVICE-tood augmented\application	Context setLoading
